In [1]:
import pandas as pd
import os
import re
from Secrets.NTE_BMN_sites import sites

In [2]:
file = input("Enter the name of the file: ")

In [3]:
# Read the CSV file into a dataframe
df = pd.read_csv(file)

#### Checking Column Names for Accuracy

In [4]:
correct_cols = [
    "Camera Name",
    "IP Address",
    "Post Deployment Status",
    "Network Fabric",
    "Building",
    "Platform",
]

# Get a list of the current column names
current_cols = df.columns.tolist()

# Loop through the current columns
for col in current_cols:
    # Check if the column name is in the correct names list
    if col in correct_cols:
        continue
    # Check if a similar named column exists in the correct names list
    for correct_col in correct_cols:
        if col.lower() == correct_col.lower():
            df.rename(columns={col: correct_col}, inplace=True)
            break


#### Checks if Building is present then puts in the correct name

In [5]:
df["Building"] = df["Building"].fillna(df["Camera Name"].str.extract(r"([A-Z]{3}\d{1,3})", expand=False))

#### converting uc_ to "UNDER_CONSTRUCTION" in Camera Names column

In [6]:
df.loc[
    df["Camera Name"].str.startswith("uc_"), "Post Deployment Status"
] = "UNDER_CONSTRUCTION"

In [7]:
df["Camera Name"] = df["Camera Name"].apply(
    lambda x: x.replace("uc_", "") if x.startswith("uc_") else x
)

In [8]:
if 'Platform' not in df.columns:
    df['Platform'] = 'CVR'
else:
    df.loc[df.notna().any(axis=1), 'Platform'] = 'CVR'


#### Keeping only the BMN sites IPs

In [9]:
df = df[df['Building'].isin(sites)]
df = df[df["IP Address"].str.startswith("10.")]
df = df.drop(df[~df["IP Address"].str.startswith("10.")].index)


#### Gets rid of white spaces in the data

In [10]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#### Save the file as a csv with NTE Cameras added

In [11]:
building = df.iloc[1]["Building"]
match = re.search(r"[A-Z]{3}", building)
if match:
    letters = match.group(0)
else:
    letters = ""
new_filename = f"{letters} NTE Cameras"
new_path = f"{os.path.dirname(file)}/{new_filename}.csv"
df.to_csv(new_path, index=False)
display(f"File saved to {new_path}")

'File saved to ~/Downloads/TLV NTE Cameras.csv'

In [12]:
df.head()

,Building,Camera Name,IP Address,Post Deployment Status,Platform
0,TLV50,TLV50-02 RZN CAGE LEFT EMG EXIT DR,10.255.47.134,NaN,CVR
1,TLV50,TLV50-06 RZN CAGE HOT AISLE,10.255.47.138,NaN,CVR
2,TLV50,TLV50-09 RZN CAGE HOT AISLE,10.255.47.141,NaN,CVR
3,TLV50,TLV50-11 RZN CAGE REAR EMG EXIT DR,10.255.47.143,NaN,CVR
4,TLV50,TLV50-12 RZN CAGE COLD AISLE,10.255.47.144,NaN,CVR
